In [2]:
import tensorflow as tf
import numpy as np
import os
import time
from tensorflow.keras.layers import Embedding, GRU, Dense

# Load the text data
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read the text file and convert to lowercase
text = open(path_to_file, 'rb').read().decode(encoding='utf-8').lower()

# Tokenize the text into words
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([text])
text_as_int = np.array(tokenizer.texts_to_sequences([text])[0])

# Create training examples and targets
seq_length = 100
examples_per_epoch = len(text_as_int) // (seq_length + 1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Split input and target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Batch size and buffer size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

# Shuffle and batch the dataset
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Load pre-trained embeddings
embedding_dim = 100  # Adjust dimension based on pre-trained embeddings
embedding_matrix = np.random.random((len(tokenizer.word_index) + 1, embedding_dim))  # Random initialization
# Load pre-trained embeddings (e.g., GloVe)
# embedding_matrix = load_pretrained_embeddings()

# Build the model
vocab_size = len(tokenizer.word_index) + 1

model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False,
              batch_input_shape=[BATCH_SIZE, None]),
    GRU(256, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    Dense(vocab_size)
])

# Define loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Train the model
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Generate text
def generate_text(model, start_string):
    num_generate = 1000
    input_eval = tokenizer.texts_to_sequences([start_string])[0]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([[predicted_id]], 0)
        text_generated.append(tokenizer.index_word[predicted_id])

    return (start_string + ' '.join(text_generated))

# Restore the latest checkpoint
model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False,
              batch_input_shape=[1, None]),
    GRU(256, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    Dense(vocab_size)
])
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

print(generate_text(model, start_string="romeo"))


Epoch 1/20
31/31 [==============================] - 119s 4s/step - loss: 7.5362
Epoch 2/20
31/31 [==============================] - 115s 4s/step - loss: 6.8337
Epoch 3/20
31/31 [==============================] - 118s 4s/step - loss: 6.8151
Epoch 4/20
31/31 [==============================] - 109s 4s/step - loss: 6.8126
Epoch 5/20
31/31 [==============================] - 110s 4s/step - loss: 6.8032
Epoch 6/20
31/31 [==============================] - 110s 4s/step - loss: 6.7916
Epoch 7/20
31/31 [==============================] - 110s 4s/step - loss: 6.7752
Epoch 8/20
31/31 [==============================] - 116s 4s/step - loss: 6.7549
Epoch 9/20
31/31 [==============================] - 108s 3s/step - loss: 6.7309
Epoch 10/20
31/31 [==============================] - 109s 3s/step - loss: 6.7026
Epoch 11/20
31/31 [==============================] - 109s 3s/step - loss: 6.6709
Epoch 12/20
31/31 [==============================] - 109s 4s/step - loss: 6.6406
Epoch 13/20
31/31 [==================

In [3]:
def generate_text(model, tokenizer, start_string, num_generate=1000):
    input_eval = tokenizer.texts_to_sequences([start_string])[0]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    model.reset_states()
    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([[predicted_id]], 0)
        text_generated.append(tokenizer.index_word[predicted_id])

    return start_string + ' '.join(text_generated)

# Load the model and tokenizer
# Assuming model and tokenizer are already loaded

# Interactive loop
while True:
    user_input = input("Enter starting string (or type 'quit' to exit): ")
    if user_input.lower() == 'quit':
        print("Exiting...")
        break
    generated_text = generate_text(model, tokenizer, start_string=user_input)
    print("Generated text:")
    print(generated_text)


Enter starting string (or type 'quit' to exit): beautiful
Generated text:
beautifulforswear vincentio all gremio i be words the crown of york is reign lodgers to myself sith despite hath hath water i'll the that i and could rather on his deceased is blood juliet he he which nurse my tricks the goodly not are will villains be god win be like full thou nobility to all thou war our though massacre pleasure good die spiritual distressed in our news by this love how well not henry prolixious this might my side caps capulet but no an it gentlemen isabella speak at well thee for think i ha which amen marry the bare covenants i sing to and is as with battle will be i will here hope tut a comfort entreat of thither field dyed i'll you are were of you departure he the red of the place has room he in his put base not in tell clarence your my lord is clarence full grey the great dead catesby leave madam the his and hose the i' in a poor first by deceive fall thy sick petruchio concluded his noble 